In [ ]:
import pandas as pd
import sqlite3
from ipywidgets import HTML
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
from collections import defaultdict



In [ ]:
def rename_feature(feature_id, mag_names):
    for mag in mag_names:
        if feature_id.startswith(mag):
            return feature_id.replace(mag, mag_names[mag])
    else:
        return feature_id
    
def HArrow(start, end, y, width, head_length=0, **kwargs):
    dx = end - start
    assert head_length < np.abs(dx), "Head length too long. {} > {}".format(head_length, np.abs(dx))
    return mpl.patches.FancyArrow(x=start, y=y, dx=dx, dy=0, width=width, head_length=head_length,
                                  length_includes_head=True, **kwargs)

def color_map_from_labels(labels, cm, default_color=None):
    labels = set(labels)
    if isinstance(cm, mpl.colors.Colormap):
        cm = defaultdict(lambda: default_color, zip(labels, cm(np.linspace(0, 1, len(labels)))))
    return cm

def plot_genomic_region(feature_df, domain_df=None,
                        feature_colorby='feature_colorby',
                        feature_annotate=None,
                        feature_cm=None, 
                        domain_colorby='domain_colorby',
                        domain_cm=None,
                        ax=None,
                        arrowhead_aspect=0.002,
                        add_legend=True,
                        feature_legend='Features',
                        domain_legend='Domains'
                       ):
    """
    
    feature_df[['feature_id', 'feature_start', 'feature_stop', *feature_colorby*, *feature_annotate*]]
    domain_df[['domain_start', 'domain_stop', *domain_colorby*]]

    """
    if domain_df is None:
        domain_df = pd.DataFrame([], columns=['domain_id', 'domain_start', 'domain_stop', domain_colorby])
        
    if not feature_cm:
        feature_cm = mpl.cm.Paired
    if not domain_cm:
        domain_cm = mpl.cm.Set3
    feature_cm = color_map_from_labels(feature_df[feature_colorby].dropna(), feature_cm)
    domain_cm = color_map_from_labels(domain_df[domain_colorby].dropna(), domain_cm)
    
    if not ax:
        fig, ax = plt.subplots(figsize=(16, 4))
        
    region_left = feature_df[['feature_start', 'feature_stop']].min().min()
    region_right = feature_df[['feature_start', 'feature_stop']].max().max()
    arrow_width = 0.6
    region_width = region_right - region_left
    head_length = region_width * arrowhead_aspect / arrow_width
    ax.set_xlim(region_left - 4 * head_length,
                region_right + 4 * head_length)
    ax.set_ylim(-1, 1)
    ax.set_yticks([])
    
    ax.plot([region_left, region_right], [0, 0], color='k')
    
    for _, ft in feature_df.iterrows():
        ax.add_patch(HArrow(ft.feature_start, ft.feature_stop,
                            arrow_width * ft.strand * 1/2,
                            width=arrow_width, head_width=arrow_width,
                            head_length=head_length,
                            facecolor=feature_cm[ft[feature_colorby]],
                            edgecolor='black'))
        ax.annotate(ft.feature_id,
                    (ft[['feature_start', 'feature_stop']].mean(),
                     (arrow_width + 0.05) * ft.strand),
                    ha='center', va='center', fontweight='bold',
                    fontsize=8)
        if feature_annotate:
            ax.annotate(ft[feature_annotate],
                        (ft[['feature_start', 'feature_stop']].mean(),
                         arrow_width * ft.strand * 1/2),
                        ha='center', va='center', rotation=-90,
                        color='black')
        
    for _, dm in domain_df.iterrows():
        ax.add_patch(HArrow(dm.domain_start, dm.domain_stop,
                            arrow_width * 1/8 * dm.strand,
                            width=arrow_width / 4,
                            head_width=arrow_width / 4,
                            head_length=0,
                            facecolor=domain_cm[dm[domain_colorby]]))    

    if feature_legend:
        feature_artists = []
        for lbl in feature_cm:
            if lbl is None:
                continue
            feature_artists.append(ax.add_patch(HArrow(-1, -2, 0, 2, facecolor=feature_cm[lbl], label=lbl)))
        feature_legend = ax.legend(handles=feature_artists, loc='lower left', title=feature_legend)
        ax.add_artist(feature_legend)
        
    if domain_legend:
        domain_artists = []
        for lbl in domain_cm:
            if lbl is None:
                continue
            domain_artists.append(ax.add_patch(HArrow(-1, -2, 0, 2, facecolor=domain_cm[lbl], label=lbl)))
        domain_legend = ax.legend(handles=domain_artists, loc='upper right', title=domain_legend)
        ax.add_artist(domain_legend)


In [ ]:
color_map = {
    'GH13': 'darkblue'
  , 'CBM20': 'darkgreen'
  , 'GH97': 'lightblue'
  , 'GH31': 'lightpink'
  , 'GH57': 'lightpink'
  , 'GH70': 'lightpink'
  , 'GH77': 'lightpink'
  , 'CBM69': 'lightgreen'
  , 'CBM26': 'plum'
  , 'GH66': 'khaki'
}

In [ ]:
con = sqlite3.connect('data/core.muri.2.denorm.db')

In [ ]:
mag = pd.read_sql("""
    SELECT
        mag_id
      , mag_class
      , completeness
      , contamination
      , n_contigs
      , total_length
      , gc_percent
      , n50
    FROM mag
    JOIN checkm USING (mag_id)
    JOIN quast USING (mag_id)
                  """, con=con,
                  index_col=['mag_id'])

mag_names = { 'Otu0001_vC': 'B1-A'
            , 'Otu0001_vB': 'B1-B'
            , 'Otu0007_vA': 'B2'
            , 'Otu0004_vA': 'B3'
            , 'Otu0005_vA': 'B4'
            , 'Otu0009_vA': 'B5'
            , 'Otu0017_vA': 'B6'
            , 'Otu0049_vA': 'B7'
            }

In [ ]:
pul = pd.read_sql("""
    SELECT
        seed_id
      , sequence_id
      , feature_id
      , fd.left AS feature_start
      , fd.right AS feature_stop
      , susC
      , susD
      , susEF
      , susG
      , domain_id
      , cmd.left AS domain_start_aa
      , cmd.right AS domain_stop_aa
      , localization
      , opf_id
      , architecture
      , product_description
    FROM feature_neighborhood
    JOIN feature_details AS fd USING (feature_id)
    LEFT JOIN (SELECT feature_id, 1 AS susC FROM putative_susC) AS c USING (feature_id)
    LEFT JOIN (SELECT feature_id, 1 AS susD FROM putative_susD) AS d USING (feature_id)
    LEFT JOIN (SELECT feature_id, 1 AS susEF FROM putative_susEF) AS e USING (feature_id)
    LEFT JOIN (SELECT feature_id, 1 AS susG FROM putative_susG) AS g USING (feature_id)
    LEFT JOIN feature_x_cazy_minimal_domain AS cmd USING (feature_id)
    WHERE distance < 25000
      AND seed_id IN (SELECT feature_id FROM putative_PUL_susC)
                  """, con=con)

pul['feature_left'] = pul[['feature_start', 'feature_stop']].min(1)
pul['feature_right'] = pul[['feature_start', 'feature_stop']].max(1)
pul['strand'] = (pul.feature_start < pul.feature_stop).map({True: 1, False: -1})
pul['domain_start'] = pul.feature_start + (pul.domain_start_aa * 3 * pul.strand)
pul['domain_stop'] = pul.feature_start + (pul.domain_stop_aa * 3 * pul.strand)
pul[['susC', 'susD', 'susEF', 'susG']] = pul[['susC', 'susD', 'susEF', 'susG']].fillna(0).astype(bool)

In [ ]:
sequence_id = 'Otu0001_vC_5_pilon_0_71909'
region_left = 47413
region_right = 60077

d = pul[ (pul.sequence_id == sequence_id)
       & (pul.feature_right > region_left)
       & (pul.feature_left < region_right)
       ].drop(['seed_id'], axis=1).drop_duplicates()

fig, ax = plt.subplots(figsize=(15, 5))

arrow_width = 0.5
region_width = region_right - region_left
head_length = region_width / (arrow_width * 500)

ax.set_xlim(region_left, region_right)
ax.set_ylim(-1, 1)
ax.set_yticks([])
ax.set_title(sequence_id)

# Plot line
ax.plot([region_left, region_right], [0, 0], color='k')

# Draw features
for _, ft in d[['feature_id', 'feature_start', 'feature_stop',
                'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates().iterrows():
    # Pick color
    if ft.susC:
        c = 'purple'
    elif ft.susD:
        c = 'blue'
    elif ft.susEF:
        c = 'green'
    else:
        c = 'grey'
    ax.add_patch(HArrow(ft.feature_start, ft.feature_stop,
                        arrow_width * ft.strand * 1/2,
                        width=arrow_width, head_width=arrow_width,
                        head_length=head_length,
                        facecolor=c, edgecolor='black'))
    ax.annotate('{}\n({})'.format(ft.opf_id, ft.localization),
                (ft[['feature_start', 'feature_stop']].mean(),
                arrow_width * ft.strand * 1/2),
                ha='center', va='center', rotation=-90,
                color='white', fontweight='bold')

# Feature legend
for sus_analogue in ['susC', 'susD', 'susEF']:
    if d[sus_analogue].sum() > 0:
        if sus_analogue == 'susC':
            c = 'purple'
        elif sus_analogue == 'susD':
            c = 'blue'
        elif sus_analogue == 'susEF':
            c = 'green'
        else:
            continue
        ax.add_patch(HArrow(-1, -2, 0, 2, facecolor=c, label='{}-like'.format(sus_analogue)))

# Draw domains
previous_domains = set()
for _, dm in (d[['feature_id', 'domain_id',
                 'domain_start', 'domain_stop',
                 'strand']]
               .dropna().drop_duplicates().iterrows()):
    if dm.domain_id in color_map:
        c = color_map[dm.domain_id]
        if dm.domain_id in previous_domains:
            label='__nolegend__'
        else:
            label=dm.domain_id
            previous_domains |= {dm.domain_id}
    else:
        c = 'lightgrey'
        label='__nolegend__'
    ax.add_patch(HArrow(dm.domain_start, dm.domain_stop,
                        arrow_width * 1/8 * dm.strand,
                        width=arrow_width / 4,
                        head_width=arrow_width / 4,
                        head_length=0,
                        facecolor=c, label=label))

ax.legend()

## Invert X axis and Y axis.
#ax.invert_xaxis()
#ax.invert_yaxis()

feats = ['feature_id', 'strand', 'feature_start', 'feature_stop',
         'domain_start', 'domain_stop', 'susC', 'susD', 'susEF',
         'susG', 'domain_id', 'localization', 'opf_id',
         'architecture', 'product_description']
d.fillna('')[feats]

In [ ]:
for sequence_id, region_left, region_right in [('Otu0001_vC_4_pilon_0_79258', 54155, 67869),
                                               ('Otu0001_vB_12_pilon_0_69629', 42592, 56306),
                                               
                                               ('Otu0001_vC_5_pilon_0_71909', 6009, 23280),
                                               ('Otu0001_vB_3_pilon_0_130094', 450, 17727),
                                               
                                               ('Otu0001_vC_5_pilon_0_71909', 46113, 72326),
                                               ('Otu0001_vB_3_pilon_0_130094', 41228, 66535),

                                               ('core-k161_1293133_pilon_0_116895', 60922, 77513),
                                               ('Otu0009_vA_23_pilon_445_51479', 2760, 17778)]:
    d = pul[ (pul.sequence_id == sequence_id)
           & (pul.feature_right > region_left)
           & (pul.feature_left < region_right)
           ].drop(['seed_id'], axis=1).drop_duplicates()

    fig, ax = plt.subplots(figsize=(15, 5))

    arrow_width = 0.5
    region_width = region_right - region_left
    head_length = region_width / (arrow_width * 500)

    ax.set_xlim(region_left, region_right)
    ax.set_ylim(-1, 1)
    ax.set_yticks([])
    ax.set_title(sequence_id)

    # Plot line
    ax.plot([region_left, region_right], [0, 0], color='k')

    # Draw features
    for _, ft in d[['feature_id', 'feature_start', 'feature_stop',
                    'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates().iterrows():
        # Pick color
        if ft.susC:
            c = 'purple'
        elif ft.susD:
            c = 'blue'
        elif ft.susEF:
            c = 'green'
        else:
            c = 'grey'
        ax.add_patch(HArrow(ft.feature_start, ft.feature_stop,
                            arrow_width * ft.strand * 1/2,
                            width=arrow_width, head_width=arrow_width,
                            head_length=head_length,
                            facecolor=c, edgecolor='black'))
        ax.annotate('{}\n({})'.format(ft.opf_id, ft.localization),
                    (ft[['feature_start', 'feature_stop']].mean(),
                     arrow_width * ft.strand * 1/2),
                    ha='center', va='center', rotation=-90,
                    color='white', fontweight='bold')
        ax.annotate(ft.feature_id,
                    (ft[['feature_start', 'feature_stop']].mean(),
                     (arrow_width + 0.05) * ft.strand),
                    ha='center', va='center', fontweight='bold',
                    fontsize=8)

    # Feature legend
    for sus_analogue in ['susC', 'susD', 'susEF']:
        if d[sus_analogue].sum() > 0:
            if sus_analogue == 'susC':
                c = 'purple'
            elif sus_analogue == 'susD':
                c = 'blue'
            elif sus_analogue == 'susEF':
                c = 'green'
            else:
                continue
            ax.add_patch(HArrow(-1, -2, 0, 2, facecolor=c, label='{}-like'.format(sus_analogue)))

    # Draw domains
    previous_domains = set()
    for _, dm in (d[['feature_id', 'domain_id',
                     'domain_start', 'domain_stop',
                     'strand']]
                   .dropna().drop_duplicates().iterrows()):
        if dm.domain_id in color_map:
            c = color_map[dm.domain_id]
            if dm.domain_id in previous_domains:
                label='__nolegend__'
            else:
                label=dm.domain_id
                previous_domains |= {dm.domain_id}
        else:
#            print(dm.domain_id)
            c = 'lightgrey'
            label='__nolegend__'
        ax.add_patch(HArrow(dm.domain_start, dm.domain_stop,
                            arrow_width * 1/8 * dm.strand,
                            width=arrow_width / 4,
                            head_width=arrow_width / 4,
                            head_length=0,
                            facecolor=c, label=label))

    ax.legend()

    ## Invert X axis and Y axis.
    #ax.invert_xaxis()
    #ax.invert_yaxis()

In [ ]:
sequence_id, region_left, region_right = ('core-k161_1293133_pilon_0_116895', 60922, 77513)
d = pul[ (pul.sequence_id == sequence_id)
       & (pul.feature_right > region_left)
       & (pul.feature_left < region_right)
       ].drop(['seed_id'], axis=1).drop_duplicates()

fig, ax = plt.subplots(figsize=(20, 5))

arrow_width = 0.5
region_width = region_right - region_left
head_length = region_width / (arrow_width * 500)

ax.set_xlim(region_left, region_right)
ax.set_ylim(-1, 1)
ax.set_yticks([])
ax.set_title(sequence_id)

# Plot line
ax.plot([region_left, region_right], [0, 0], color='k')

# Draw features
for _, ft in d[['feature_id', 'feature_start', 'feature_stop',
                'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates().iterrows():
    # Pick color
    if ft.susC:
        c = 'purple'
    elif ft.susD:
        c = 'blue'
    elif ft.susEF:
        c = 'green'
    else:
        c = 'grey'
    ax.add_patch(HArrow(ft.feature_start, ft.feature_stop,
                        arrow_width * ft.strand * 1/2,
                        width=arrow_width, head_width=arrow_width,
                        head_length=head_length,
                        facecolor=c, edgecolor='black'))
    ax.annotate('{}\n({})'.format(ft.opf_id, ft.localization),
                (ft[['feature_start', 'feature_stop']].mean(),
                 arrow_width * ft.strand * 1/2),
                ha='center', va='center', rotation=-90,
                color='white', fontweight='bold')
    ax.annotate(ft.feature_id,
                (ft[['feature_start', 'feature_stop']].mean(),
                 (arrow_width + 0.05) * ft.strand),
                ha='center', va='center', fontweight='bold',
                fontsize=8)

# Feature legend
for sus_analogue in ['susC', 'susD', 'susEF']:
    if d[sus_analogue].sum() > 0:
        if sus_analogue == 'susC':
            c = 'purple'
        elif sus_analogue == 'susD':
            c = 'blue'
        elif sus_analogue == 'susEF':
            c = 'green'
        else:
            continue
        ax.add_patch(HArrow(-1, -2, 0, 2, facecolor=c, label='{}-like'.format(sus_analogue)))

# Draw domains
previous_domains = set()
for _, dm in (d[['feature_id', 'domain_id',
                 'domain_start', 'domain_stop',
                 'strand']]
               .dropna().drop_duplicates().iterrows()):
    if dm.domain_id in color_map:
        c = color_map[dm.domain_id]
        if dm.domain_id in previous_domains:
            label='__nolegend__'
        else:
            label=dm.domain_id
            previous_domains |= {dm.domain_id}
    else:
#            print(dm.domain_id)
        c = 'lightgrey'
        label='__nolegend__'
    ax.add_patch(HArrow(dm.domain_start, dm.domain_stop,
                        arrow_width * 1/8 * dm.strand,
                        width=arrow_width / 4,
                        head_width=arrow_width / 4,
                        head_length=0,
                        facecolor=c, label=label))

ax.legend()

## Invert X axis and Y axis.
#ax.invert_xaxis()
#ax.invert_yaxis()

fig.savefig('/Users/bjsmith/Desktop/otu7_starch_pul.pdf')

In [ ]:
fig, axs = plt.subplots(2, figsize=(20, 10))

for mag_id, sequence_id, region_left, region_right, ax in [ ('Otu0007_vA', 'core-k161_1293133_pilon_0_116895', 60922, 77513, axs[0])
                                                          , ('Bacteroides_ovatus_ATCC_8483', 'CP012938.1', 3977866, 3992117, axs[1])
                                                          ]:
    d = pul[ (pul.sequence_id == sequence_id)
           & (pul.feature_right > region_left)
           & (pul.feature_left < region_right)
           ].drop(['seed_id'], axis=1).drop_duplicates()


    arrow_width = 0.5
    region_width = region_right - region_left
    head_length = region_width / (arrow_width * 500)

    ax.set_xlim(region_left, region_right)
    ax.set_ylim(-1, 1)
    ax.set_yticks([])
    ax.set_title(mag_id)

    # Plot line
    ax.plot([region_left, region_right], [0, 0], color='k')

    # Draw features
    for _, ft in d[['feature_id', 'feature_start', 'feature_stop',
                    'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates().iterrows():
        # Pick color
        if ft.susC:
            c = 'powderblue'
        elif ft.susD:
            c = 'cornflowerblue'
        elif ft.susEF:
            c = 'tan'
        else:
            c = 'grey'
        ax.add_patch(HArrow(ft.feature_start, ft.feature_stop,
                            arrow_width * ft.strand * 1/2,
                            width=arrow_width, head_width=arrow_width,
                            head_length=head_length,
                            facecolor=c, edgecolor='black'))
        ax.annotate('{}\n({})'.format(ft.opf_id, ft.localization),
                    (ft[['feature_start', 'feature_stop']].mean(),
                     arrow_width * ft.strand * 1/2),
                    ha='center', va='center', rotation=-90)
        ax.annotate(ft.feature_id.replace(mag_id + '_', ''),
                    (ft[['feature_start', 'feature_stop']].mean(),
                     (arrow_width + 0.05) * ft.strand),
                    ha='center', va='center', fontweight='bold',
                    fontsize=8)

    # Feature legend
    for sus_analogue in ['susC', 'susD', 'susEF']:
        if d[sus_analogue].sum() > 0:
            if sus_analogue == 'susC':
                c = 'powderblue'
            elif sus_analogue == 'susD':
                c = 'cornflowerblue'
            elif sus_analogue == 'susEF':
                c = 'tan'
            else:
                continue
            ax.add_patch(HArrow(-1, -2, 0, 2, facecolor=c, label='{}-like'.format(sus_analogue)))

    # Draw domains
    previous_domains = set()
    for _, dm in (d[['feature_id', 'domain_id',
                     'domain_start', 'domain_stop',
                     'strand']]
                   .dropna().drop_duplicates().iterrows()):
        if dm.domain_id in color_map:
            c = color_map[dm.domain_id]
            if dm.domain_id in previous_domains:
                label='__nolegend__'
            else:
                label=dm.domain_id
                previous_domains |= {dm.domain_id}
        else:
            print(dm.domain_id)
            c = 'lightgrey'
            label='__nolegend__'
        ax.add_patch(HArrow(dm.domain_start, dm.domain_stop,
                            arrow_width * 1/8 * dm.strand,
                            width=arrow_width / 4,
                            head_width=arrow_width / 4,
                            head_length=0,
                            facecolor=c, label=label))

    ax.legend()

    # Invert X axis and Y axis.
    ax.invert_xaxis()
    ax.invert_yaxis()

fig.savefig('/Users/bjsmith/Desktop/otu7_starch_pul.pdf')

In [ ]:
sequence_id, region_left, region_right = ('core-k161_1293133_pilon_0_116895', 60922, 77513)
d = pul[ (pul.sequence_id == sequence_id)
       & (pul.feature_right > region_left)
       & (pul.feature_left < region_right)
       ].drop(['seed_id'], axis=1).drop_duplicates()

f = d[['feature_id', 'feature_start', 'feature_stop', 'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates()

f['feature_colorby'] = None
f.loc[lambda x: x.susEF, 'feature_colorby'] = 'susEF'
f.loc[lambda x: x.susD, 'feature_colorby'] = 'susD'
f.loc[lambda x: x.susC, 'feature_colorby'] = 'susC'
f['annotate'] = f.apply(lambda x: '{}\n({})'.format(x.opf_id, x.localization), axis=1)
f.feature_id = f.feature_id.str.replace('Otu0007_vA_', '')

plot_genomic_region(f, d.dropna(subset=['domain_id']),
                    feature_annotate='annotate',
                    domain_colorby='domain_id',
                    feature_legend_title='sus homologues',
                    domain_legend_title='CAZy domains')

In [ ]:
fig, axs = plt.subplots(2, figsize=(16, 8))

regions = [ ('Otu0007_vA', 'core-k161_1293133_pilon_0_116895', 60922, 77513, axs[0])
          , ('Bacteroides_ovatus_ATCC_8483', 'CP012938.1', 3977866, 3992117, axs[1])
          ]

for mag_id, sequence_id, region_left, region_right, ax in regions:
    d = pul[ (pul.sequence_id == sequence_id)
           & (pul.feature_right > region_left)
           & (pul.feature_left < region_right)
           ].drop(['seed_id'], axis=1).drop_duplicates()
    f = d[['feature_id', 'feature_start', 'feature_stop', 'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates()
    f['feature_colorby'] = None
    f.loc[lambda x: x.susEF, 'feature_colorby'] = 'susEF'
    f.loc[lambda x: x.susD, 'feature_colorby'] = 'susD'
    f.loc[lambda x: x.susC, 'feature_colorby'] = 'susC'
    f['annotate'] = f.apply(lambda x: '{}\n({})'.format(x.opf_id, x.localization), axis=1)
    f.feature_id = f.feature_id.str.replace(mag_id + '_', '')
    plot_genomic_region(f, d.dropna(subset=['domain_id']),
                        feature_annotate='annotate',
                        domain_colorby='domain_id',
                        feature_legend_title='sus homologues',
                        domain_legend_title='CAZy domains', ax=ax)

In [ ]:
domain_cmap = color_map_from_labels(['GH13', 'GH31', 'GH97', 'GH66', 'CBM20', 'CBM69', 'CBM26'],
                                    mpl.cm.Set3, default_color='lightgrey')
feature_cmap = color_map_from_labels(['susC', 'susD', 'susEF'],
                                     mpl.cm.Paired, default_color='lightgrey')

regions = [ ('Otu0001_vC', 'Otu0001_vC_4_pilon_0_79258', 54155, 67869)
          , ('Otu0001_vC', 'Otu0001_vC_5_pilon_0_71909', 46113, 72326)
          , ('Otu0001_vC', 'Otu0001_vC_5_pilon_0_71909', 6009, 23280)
#          , ('Otu0001_vB', 'Otu0001_vB_3_pilon_0_130094', 450, 17727)
#
#          , ('Otu0001_vB', 'Otu0001_vB_3_pilon_0_130094', 41228, 66535)
#
          , ('Otu0007_vA', 'core-k161_1293133_pilon_0_116895', 60922, 77513)
          , ('Otu0009_vA', 'Otu0009_vA_23_pilon_445_51479', 2760, 17778)
#          , ('Bacteroides_ovatus_ATCC_8483', 'CP012938.1', 3977866, 3992117)
          ]

fig, axs = plt.subplots(len(regions), figsize=(16, 4 * len(regions)))

for (mag_id, sequence_id, region_left, region_right), ax in zip(regions, axs):
    d = pul[ (pul.sequence_id == sequence_id)
           & (pul.feature_right > region_left)
           & (pul.feature_left < region_right)
           ].drop(['seed_id'], axis=1).drop_duplicates()
    f = d[['feature_id', 'feature_start', 'feature_stop', 'strand', 'susC', 'susD', 'susEF', 'opf_id', 'localization']].drop_duplicates()
    f['feature_colorby'] = None
    f.loc[lambda x: x.susEF, 'feature_colorby'] = 'susEF'
    f.loc[lambda x: x.susD, 'feature_colorby'] = 'susD'
    f.loc[lambda x: x.susC, 'feature_colorby'] = 'susC'
    f['annotate'] = f.apply(lambda x: '{}\n({})'.format(x.opf_id, x.localization), axis=1)
    f.feature_id = f.feature_id.str.replace(mag_id + '_', '')
    plot_genomic_region(f, d.dropna(subset=['domain_id']),
                        feature_annotate='annotate',
                        domain_colorby='domain_id',
                        domain_cm=domain_cmap,
                        feature_cm=feature_cmap,
                        feature_legend='sus-like',
                        domain_legend='cazy domains',
                        ax=ax)
    ax.set_xlim(region_left, region_right)
    if d.strand.sum() < 0:
        ax.invert_xaxis()
        ax.invert_yaxis()
    ax.set_title(mag_id)